# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.dummy import DummyClassifier

In [25]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [26]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

In [28]:
features = df.drop("is_ultra", axis = 1) #выделяем признаки
target = df['is_ultra'] #выделяем целевой признак

In [29]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, train_size=0.75, random_state=12345, stratify=target) #разбиваем данные на обучающие и тестовые

## Исследуйте модели

Модель Решающее дерево

In [30]:
param_grid = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
model = DecisionTreeClassifier()
model_tree = GridSearchCV(model, param_grid, cv=5)
model_tree.fit(features_train, target_train)
model_tree.best_score_

0.7900414937759336

Модель Случайный лес

In [31]:
param_grid = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'n_estimators': [10, 20, 30, 40, 50]}
model = RandomForestClassifier()
model_forest = GridSearchCV(model, param_grid, cv=5)
model_forest.fit(features_train, target_train)
model_forest.best_score_

0.8037344398340249

Модель Логистическая регрессия

In [32]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_log.fit(features_train, target_train)

scores_list = cross_val_score(model_log,
                              X=features_train, 
                              y=target_train, 
                              cv=5  
                              )
scores_list.mean()

0.7448132780082988

Наивысшее accuracy показала модель "Случайный лес".

## Проверьте модель на тестовой выборке

In [33]:
model_forest.score(features_test, target_test)

0.8072139303482587

Accuracy тестовой выборки близка по значению к accuracy валидационной выборки.

Проверьте модели на адекватность

Построим dummy модель по стратегии most_frequent. Т.е. dummy модель будет возвращать наиболее часто используемую метку класса. И сравним accuracy на тестовой выборке нашей модели и dummy.

In [34]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_test, target_test)

0.6940298507462687

На тестовой выборке accuracy нашей модели выше, значит она работает адекватно.